In [16]:
import json
import pandas as pd 
import sys
from collections import defaultdict


sys.path.append('..')
import ENVIRONMENT_VARIABLES as EV

In [17]:
years_analysed = [2023]

In [18]:
teams = ["Broncos", "Roosters", "Wests Tigers", "Rabbitohs", "Storm", "Eels", "Raiders", "Knights", "Dragons", "Sea Eagles", "Panthers", "Sharks", "Bulldogs", "Dolphins", "Titans", "Cowboys", "Warriors"]
variables =["Year", "Win", "Defense", "Attack", "Margin", "Home", "Versus",  "Round"]
years =  [2019, 2020, 2021, 2022, 2023, 2024, 2025]

In [19]:
player_variables =["Name"] +EV.PLAYER_LABELS

In [ ]:
# Initialize an empty dictionary to store player statistics for each year
years_arr = {}

# Open the JSON file containing player statistics data for a specific year
with open('../data/NRL/2025/NRL_player_statistics_2025.json', 'r') as file:
    data = json.load(file)  # Load JSON data from the file
    data = data['2025']['PlayerStats']  # Adjust indexing to access 'PlayerStats' key
    for year in years_analysed:  # Iterate over the years to be analyzed
        # Store player statistics data for the current year in the 'years_arr' dictionary
        years_arr[year] = data

KeyError: 0

In [ ]:
df = pd.DataFrame(columns=[f"{team} {variable}" for team in teams for variable in player_variables])

In [ ]:
# p_dfs = {}

# def custom_sort(column_name):
#     year, num = column_name.split('-')
#     return int(year), int(num)


# for team in teams:
#     p_dfs[team] = pd.DataFrame(columns=[f"{year}-{round+1}" for round in range(0, 26) for year in years_analysed])
#     p_dfs[team] = p_dfs[team][sorted(p_dfs[team].columns, key=custom_sort)]
    

In [ ]:
# team_players = {}

# for year in years_analysed:
#     for i in range(0, 26):
#         try:
#             round = years_arr[year][i]
#             round = round[str(i)]
#             for round_game in round:
#                 for game in round_game:
#                     game_split = game.split("-")
#                     game_year = game_split[0]
#                     game_round = game_split[1]
                    
#                     game_split = game.split("v")
#                     home_team = " ".join(game_split[0].split("-")[2:]).replace("-", " ").strip()
#                     away_team = " ".join(game_split[-1:]).replace("-", " ").strip()

#                     print(f"{home_team} v {away_team} - {game_year} - {game_round}")

                    
#                     players = round_game[game]
#                     player_round_stats = {} 
#                     for player in players:
#                         vals = [player[val] for val in  player_variables]
#                         player_round_stats[vals[0]] = vals[1:]
                    
#                     player_round_stats = list(player_round_stats.items())
#                     player_round_stats_home, player_round_stats_away = player_round_stats[:18], player_round_stats[18:]
#                     p_dfs[home_team][f"{year}-{i+1}"] = player_round_stats_home
#                     p_dfs[away_team][f"{year}-{i+1}"] = player_round_stats_away

                    
                    
#         except Exception as ex:
#             print(ex)


In [ ]:

player_stats = defaultdict(list)  # Dictionary to store player statistics

for year in years_analysed:  # Loop through each year to analyze
    for i in range(0, 26):  # Loop through each round (assuming there are 26 rounds)
        try:
            round = years_arr[year][i]  # Get data for a particular round
            round = round[str(i)]  # Assuming round is stored as a string key
            for round_game in round:  # Loop through each game in the round
                for game in round_game:  # Loop through each team's performance in the game
                    # Splitting game data to extract year, round, home team, and away team
                    game_split = game.split("-")
                    game_year, game_round = game_split[0], game_split[1]

                    game_split = game.split("v")
                    home_team = " ".join(game_split[0].split("-")[2:]).replace("-", " ").strip()
                    away_team = " ".join(game_split[-1:]).replace("-", " ").strip()

                    players = round_game[game]  # List of players in the game
                    player_round_stats = {}  # Dictionary to store player statistics for the game
                    for player in players:  # Loop through each player's stats
                        vals = [player[val] for val in player_variables]  # Extract required stats for each player
                        player_round_stats[vals[0]] = vals[1:]  # Store player stats with player ID as key
                    
                    player_round_stats = list(player_round_stats.items())  # Convert player stats to list of tuples
                    # Split player stats into home and away team stats
                    player_round_stats_home, player_round_stats_away = player_round_stats[:18], player_round_stats[18:]
                    
                    # Append home team player stats to player_stats dictionary
                    for player in player_round_stats_home:
                        player_stats[player[0]].append([player[1], game_round, home_team, away_team])
                    # Append away team player stats to player_stats dictionary
                    for player in player_round_stats_away:
                        player_stats[player[0]].append([player[1], game_round, away_team, home_team])
        except IndexError as ex:
            print(f"Run out of round data, round: {i}, \n{ex}")
                    
        except Exception as ex:
            print(f"{ex}\tround: {i}:{game}")  # Print any exceptions that occur during processing


In [ ]:
headers

In [ ]:
# Combine player labels with additional information
headers = EV.PLAYER_LABELS + ["Round", "Team", "Opposition"]

# Loop through each player and their statistics
for player, values in player_stats.items():
    # Combine round statistics with additional information
    player_stat = [[*round[0], *round[1:]] for round in values]
    # Create a DataFrame from the combined statistics and headers
    df = pd.DataFrame(player_stat, columns=headers)

    # Save the DataFrame as both text and CSV files with player's ID as the filename
    df.to_csv(f"../data/txt/players/txt/{player}.txt", encoding='utf-8', sep='\t', index=False)
    df.to_csv(f"../data/txt/players/csv/{player}.csv", encoding='utf-8', sep='\t', index=False)

